In [1]:
import numpy as np
from torch.nn import Conv2d
from torch import from_numpy
from numpy.testing import assert_array_equal

In [2]:
def conv2d(input, weight, bias, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1,
                   bias_enabled=True, padding_mode='zeros'):
    batches = len(input)
    out = []

    for b in range(batches):
        h_in, w_in = (input[b]).shape[1], (input[b]).shape[2]

        kernel_size = (kernel_size, kernel_size) if not isinstance(kernel_size, tuple) else kernel_size
        stride = (stride, stride) if not isinstance(stride, tuple) else stride
        dilation = (dilation, dilation) if not isinstance(dilation, tuple) else dilation

        padding = padding if isinstance(padding, tuple) else (padding, padding)

        h_out = int((h_in + 2 * padding[0] - dilation[0] * (kernel_size[0] - 1) - 1) / stride[0] + 1)
        w_out = int((w_in + 2 * padding[1] - dilation[1] * (kernel_size[1] - 1) - 1) / stride[1] + 1)

        out.append(np.zeros((out_channels, h_out, w_out)))

        for c_out in range(out_channels):
            for y_out in range(h_out):
                for x_out in range(w_out):
                    summation = 0
                    for c_in in range(in_channels):
                        for kernel_y in range(kernel_size[0]):
                            for kernel_x in range(kernel_size[1]):
                                y_in = y_out * stride[0] + kernel_y * dilation[0] - padding[0]
                                x_in = x_out * stride[1] + kernel_x * dilation[1] - padding[1]
                                if 0 <= y_in < h_in and 0 <= x_in < w_in:
                                    summation += input[b][c_in][y_in][x_in] * weight[c_out][c_in][kernel_y][kernel_x]
                                elif padding_mode == 'replicate':
                                    y_in = max(0, min(y_in, h_in - 1))
                                    x_in = max(0, min(x_in, w_in - 1))
                                    summation += input[b][c_in][y_in][x_in] * weight[c_out][c_in][kernel_y][kernel_x]

                    out[b][c_out][y_out][x_out] = summation + (bias[c_out] if bias_enabled else 0)

    return np.array(out)

In [4]:
def test(own, torch, input, weight, bias):
    torch.weight.data = from_numpy(weight).float()
    torch.bias.data = from_numpy(bias).float()
    own_result = np.floor(own)
    torch_result = np.floor(torch(from_numpy(input).float()).detach().numpy())
    #print(np.array(own_result).shape)
    #print(np.array(torch_result).shape)
    print("custom")
    print(own_result)
    print("pytorch")
    print(torch_result)
    assert_array_equal(own_result, torch_result)

print("Нулевые входа и веса")
inp, weight, bias = np.zeros((1, 1, 4, 4)), np.zeros((2, 1, 2, 2)), np.zeros(2)
own_result = conv2d(inp, weight, bias, in_channels=1, out_channels=2, kernel_size=2)
torch_result = Conv2d(in_channels=1, out_channels=2, kernel_size=2)
test(own_result, torch_result, inp, weight, bias)
    
inp = np.array([[[[1, 2, 3, 4], [9, 10, 11, 12], [5, 6, 7, 8],[13, 14, 15, 16]],[[1, 2, 3, 4], [9, 10, 11, 12], [5, 6, 7, 8],[13, 14, 15, 16]]]])
weight = np.array([[[[1, 1],[1, 1]],[[1, 1],[1, 1]]], [[[1, 1],[1, 1]], [[1, 1],[1, 1]]]])
bias = np.array([0, 0])
print("Стандартные данные")
own_result = conv2d(inp, weight, bias, in_channels=2, out_channels=2, kernel_size=2)
torch_result = Conv2d(in_channels=2, out_channels=2, kernel_size=2)
test(own_result, torch_result, inp, weight, bias)

print("Случайные данные")
np.random.seed(42)
inp = np.random.rand(1, 2, 5, 5) 
weight = np.random.rand(2, 2, 2, 2)
bias = np.random.rand(2)
own_result = conv2d(inp, weight, bias, in_channels=2, out_channels=2, kernel_size=2)
torch_result = Conv2d(in_channels=2, out_channels=2, kernel_size=2)
test(own_result, torch_result, inp, weight, bias)

inp = np.array([[[[1, 2, 3, 4], [9, 10, 11, 12], [5, 6, 7, 8],[13, 14, 15, 16]],[[1, 2, 3, 4], [9, 10, 11, 12], [5, 6, 7, 8],[13, 14, 15, 16]]]])
weight = np.array([[[[1, 1],[1, 1]],[[1, 1],[1, 1]]], [[[1, 1],[1, 1]], [[1, 1],[1, 1]]]])
bias = np.array([0, 0])
    
print("с параметром bias")
own_result = conv2d(inp, weight, bias=[1,1], in_channels=2, out_channels=2, kernel_size=2)
torch_result = Conv2d(in_channels=2, out_channels=2, kernel_size=2)
test(own_result, torch_result, inp, weight, bias=np.array([1,1]))

print("с параметром stride")
own_result = conv2d(inp, weight, bias, in_channels=2, out_channels=2, kernel_size=2, stride=2)
torch_result = Conv2d(in_channels=2, out_channels=2, kernel_size=2, stride=2)
test(own_result, torch_result, inp, weight, bias)

print("с параметром padding")
own_result = conv2d(inp, weight, bias, in_channels=2, out_channels=2, kernel_size=2, padding=1)
torch_result = Conv2d(in_channels=2, out_channels=2, kernel_size=2, padding=1)
test(own_result, torch_result, inp, weight, bias)

print("с параметром dilation")
own_result = conv2d(inp, weight, bias, in_channels=2, out_channels=2, kernel_size=2, dilation=2)
torch_result = Conv2d(in_channels=2, out_channels=2, kernel_size=2, dilation=2)
test(own_result, torch_result, inp, weight, bias)

print("с параметром padding_mode")
own_result = conv2d(inp, weight, bias, in_channels=2, out_channels=2, kernel_size=2, padding=1, padding_mode='replicate')
torch_result = Conv2d(in_channels=2, out_channels=2, kernel_size=2, padding=1, padding_mode='replicate')
test(own_result, torch_result, inp, weight, bias)

Нулевые входа и веса
custom
[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]]
pytorch
[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]]
Стандартные данные
custom
[[[[44. 52. 60.]
   [60. 68. 76.]
   [76. 84. 92.]]

  [[44. 52. 60.]
   [60. 68. 76.]
   [76. 84. 92.]]]]
pytorch
[[[[44. 52. 60.]
   [60. 68. 76.]
   [76. 84. 92.]]

  [[44. 52. 60.]
   [60. 68. 76.]
   [76. 84. 92.]]]]
Случайные данные
custom
[[[[2. 3. 3. 2.]
   [1. 2. 3. 3.]
   [2. 2. 2. 2.]
   [1. 1. 2. 2.]]

  [[2. 1. 2. 2.]
   [2. 1. 2. 2.]
   [2. 1. 2. 2.]
   [1. 1. 1. 2.]]]]
pytorch
[[[[2. 3. 3. 2.]
   [1. 2. 3. 3.]
   [2. 2. 2. 2.]
   [1. 1. 2. 2.]]

  [[2. 1. 2. 2.]
   [2. 1. 2. 2.]
   [2. 1. 2. 2.]
   [1. 1. 1. 2.]]]]
с параметром bias
custom
[[[[45. 53. 61.]
   [61. 69. 77.]
   [77. 85. 93.]]

  [[45. 53. 61.]
   [61. 69. 77.]
   [77. 85. 93.]]]]
pytorch
[[[[45. 53. 61.]
   [61. 69. 77.]
   [77. 85. 93.]]

  [[45. 53. 61.]
   [61. 69. 77